# OpenAI Agents SDK - 고객 서비스 에이전트와 대화 관리

이 튜토리얼은 OpenAI Agents SDK를 사용하여 정교한 고객 서비스 에이전트와 프로덕션 수준의 대화 관리 시스템을 구축하는 방법을 다룬다. 영속적 메모리, 고객 정보 추출, 토큰 관리, 대화 분기/병합 등 엔터프라이즈급 대화형 에이전트 구축에 필요한 고급 패턴을 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. 고객 서비스 에이전트 메모리

이 챕터에서는 정교한 대화 관리 기능을 갖춘 고객 서비스 에이전트를 구축한다. 영속적 메모리, 고객 정보 추출, 도구 통합, 토큰 관리 등의 기능을 구현한다.

### 학습 내용

- 영속적 메모리를 갖춘 CustomerServiceAgent 클래스 구축
- 대화 턴을 통한 고객 정보 추출 및 저장
- 자동 대화 길이 및 토큰 제한 관리
- 프로덕션용 에러 핸들링 및 대화 디버깅

### 환경 설정 및 도구 정의

고객 서비스 에이전트가 사용할 도구들을 정의한다. 주문 조회, 반품 정책 확인, 지원 티켓 생성 등의 기능을 구현한다.

In [1]:
import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool
from datetime import datetime
from typing import List, Dict, Any

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

In [2]:
# 고객 서비스 도구 정의
@function_tool
def lookup_order(order_id: str) -> str:
    """주문 ID로 주문 정보를 조회한다."""
    # 데이터베이스 조회 시뮬레이션
    orders = {
        "ORD123": "주문 ORD123: 무선 헤드폰 2개, 상태: 배송중, 송장번호: TRK456789",
        "ORD456": "주문 ORD456: 노트북 1개, 상태: 처리중, 예상 배송일: 내일",
        "ORD789": "주문 ORD789: 책 3권, 상태: 배송완료, 배송완료일: 2025-06-25"
    }
    return orders.get(order_id, f"주문 {order_id}을(를) 시스템에서 찾을 수 없습니다.")

@function_tool
def check_return_policy() -> str:
    """현재 반품 정책 정보를 반환한다."""
    return "30일 반품 정책. 제품은 미사용 상태이며 원래 포장 상태여야 합니다. 불량품은 무료 반품 배송이 가능합니다."

@function_tool
def create_support_ticket(issue: str, priority: str = "medium") -> str:
    """복잡한 문제에 대한 지원 티켓을 생성한다."""
    ticket_id = f"TKT{datetime.now().strftime('%Y%m%d%H%M%S')}"
    return f"지원 티켓 {ticket_id}이(가) 생성되었습니다. 문제: {issue}. 우선순위: {priority}. 예상 응답 시간: 24-48시간."

print("고객 서비스 도구가 정의되었다: lookup_order, check_return_policy, create_support_ticket")

고객 서비스 도구가 정의되었다: lookup_order, check_return_policy, create_support_ticket


### CustomerServiceAgent 클래스

영속적 메모리와 고객 정보 추출 기능을 갖춘 고객 서비스 에이전트 클래스를 정의한다. 이 클래스는 대화 기록을 유지하고, 고객 정보를 추출하며, 토큰 사용량을 추적하고, 도구 사용을 모니터링한다.

In [3]:
class CustomerServiceAgent:
    """
    영속적 메모리와 고객 정보 추출 기능을 갖춘 고객 서비스 에이전트이다.
    """
    
    def __init__(self):
        self.agent = Agent(
            name="CustomerServiceAgent",
            instructions="""
            당신은 친절한 고객 서비스 담당자입니다.
            
            주요 행동 지침:
            - 대화 전체에서 고객 정보를 기억하세요
            - 관련이 있을 때 이전 대화 내용을 참조하세요
            - 필요할 때 도구를 사용하여 주문을 조회하고 티켓을 생성하세요
            - 친근하고 전문적이며 해결 지향적이어야 합니다
            - 문제를 해결할 수 없으면 지원 티켓을 생성하세요
            
            항상 고객을 따뜻하게 맞이하고 어떻게 도울 수 있는지 물어보세요.
            """,
            tools=[lookup_order, check_return_policy, create_support_ticket]
        )
        self.conversation_history = []
        self.customer_info = {}
    
    async def chat(self, user_message: str) -> Dict[str, Any]:
        """
        고객 메시지를 처리하고 메타데이터와 함께 응답을 반환한다.
        
        Args:
            user_message: 고객의 메시지
        
        Returns:
            응답과 메타데이터를 담은 딕셔너리
        """
        
        # 입력 구성: 대화 기록 + 새 메시지
        if self.conversation_history:
            full_input = self.conversation_history + [{"role": "user", "content": user_message}]
        else:
            full_input = user_message
        
        try:
            # 에이전트 실행
            result = await Runner.run(self.agent, full_input)
            
            # 다음 턴을 위해 대화 기록 업데이트
            self.conversation_history = result.to_input_list()
            
            # 대화에서 고객 정보 추출
            self._extract_customer_info(user_message, result.final_output)
            
            # 대화 메트릭 계산
            token_usage = self._calculate_token_usage(result)
            
            return {
                "response": result.final_output,
                "conversation_length": len(self.conversation_history),
                "customer_info": self.customer_info,
                "token_usage": token_usage,
                "tools_used": self._count_tools_used(result)
            }
            
        except Exception as e:
            print(f"응답 오류: {e}")
            # 폴백 응답 제공
            fallback_response = "죄송합니다. 기술적인 문제가 발생했습니다. 요청을 다시 말씀해주시거나 문제가 지속되면 지원팀에 연락해주세요."
            
            return {
                "response": fallback_response,
                "conversation_length": len(self.conversation_history),
                "customer_info": self.customer_info,
                "token_usage": {"input_tokens": 0, "output_tokens": 0, "total_tokens": 0},
                "tools_used": 0,
                "error": str(e)
            }
    
    def _extract_customer_info(self, user_input: str, agent_response: str):
        """대화에서 고객 정보를 추출하여 저장한다."""
        # 간단한 추출 - 프로덕션에서는 더 정교한 NLP 사용
        lower_input = user_input.lower()
        
        # 이름 추출
        if "제 이름은" in lower_input or "저는" in lower_input:
            # "제 이름은" 뒤의 이름 추출
            if "제 이름은" in lower_input:
                parts = lower_input.split("제 이름은")
                if len(parts) > 1:
                    name = parts[1].strip().split()[0]
                    self.customer_info["name"] = name
        
        # 주문 ID 추출
        if "주문" in lower_input and "ORD" in user_input:
            words = user_input.split()
            for word in words:
                if word.startswith("ORD"):
                    self.customer_info["recent_order"] = word
    
    def _calculate_token_usage(self, result) -> Dict[str, int]:
        """결과에서 총 토큰 사용량을 계산한다."""
        total_input = sum(r.usage.input_tokens for r in result.raw_responses)
        total_output = sum(r.usage.output_tokens for r in result.raw_responses)
        return {
            "input_tokens": total_input,
            "output_tokens": total_output,
            "total_tokens": total_input + total_output
        }
    
    def _count_tools_used(self, result) -> int:
        """이 턴에서 사용된 도구 수를 계산한다."""
        return len([item for item in result.new_items if item.type == "tool_call_item"])
    
    def get_conversation_summary(self) -> Dict[str, Any]:
        """전체 대화의 요약을 반환한다."""
        return {
            "total_messages": len(self.conversation_history),
            "customer_info": self.customer_info,
            "conversation_ready_for_handoff": len(self.conversation_history) > 0
        }
    
    async def manage_conversation_length(self, max_messages: int = 20):
        """토큰 제한 관리를 위해 대화가 너무 길어지면 잘라낸다."""
        if len(self.conversation_history) > max_messages:
            # 시스템 메시지와 최근 메시지 유지
            system_msg = None
            if (self.conversation_history and 
                isinstance(self.conversation_history[0], dict) and 
                self.conversation_history[0].get("role") == "system"):
                system_msg = self.conversation_history[0]
            
            # 컨텍스트 보존을 위해 최근 메시지 유지
            recent_messages = self.conversation_history[-(max_messages-2):]
            
            if system_msg:
                self.conversation_history = [system_msg] + recent_messages
            else:
                self.conversation_history = recent_messages
            
            print(f"[주의] 대화가 {len(self.conversation_history)}개 메시지로 잘렸다.")
            
            # 컨텍스트 문제를 방지하기 위해 불완전한 도구 호출 제거
            self.conversation_history = [
                msg for msg in self.conversation_history 
                if not (isinstance(msg, dict) and msg.get("role") == "assistant" and 
                       "tool_calls" in msg and not msg.get("content"))
            ]

print("CustomerServiceAgent 클래스가 정의되었다.")

CustomerServiceAgent 클래스가 정의되었다.


### 고객 서비스 대화 데모

구축한 고객 서비스 에이전트로 실제 고객 지원 시나리오를 시뮬레이션한다. 여러 턴에 걸친 대화에서 에이전트가 컨텍스트를 유지하고, 도구를 사용하며, 고객 정보를 추출하는 과정을 확인한다.

In [4]:
async def customer_service_demo():
    """
    정교한 대화 관리를 시연하는 함수이다.
    """
    
    print("고객 서비스 대화 데모")
    print("="*50)
    
    service_agent = CustomerServiceAgent()
    
    # 고객 서비스 대화 시뮬레이션
    conversation_turns = [
        "안녕하세요, 제 이름은 김민수이고 주문 관련해서 도움이 필요합니다.",
        "무선 헤드폰을 주문했는데, 주문번호는 ORD123입니다.",
        "언제 도착하나요? 다음 주에 필요해서요.",
        "사실 반품하고 싶은데요. 반품 정책이 어떻게 되나요?",
        "좋아요! 반품 절차를 시작해주실 수 있나요?",
        "감사합니다! 참조용 티켓 번호가 뭔가요?"
    ]
    
    print("고객 서비스 대화 시작...\n")
    
    for i, user_message in enumerate(conversation_turns, 1):
        print(f"[고객 - 턴 {i}] {user_message}")
        
        # 메시지 처리
        response_data = await service_agent.chat(user_message)
        
        print(f"[에이전트] {response_data['response']}")
        print(f"[메트릭] 메시지: {response_data['conversation_length']}, "
              f"토큰: {response_data['token_usage']['total_tokens']}, "
              f"도구 사용: {response_data['tools_used']}회")
        
        # 고객 정보 추출 표시
        if response_data['customer_info']:
            print(f"[고객 정보] {response_data['customer_info']}")
        
        print("-" * 50)
        
        # 대화 길이 관리 (데모를 위해 낮은 임계값 사용)
        await service_agent.manage_conversation_length(max_messages=10)
    
    # 최종 대화 요약
    summary = service_agent.get_conversation_summary()
    print("\n대화 요약:")
    print(f"  총 메시지: {summary['total_messages']}")
    print(f"  수집된 고객 정보: {summary['customer_info']}")
    print(f"  핸드오프 준비 완료: {summary['conversation_ready_for_handoff']}")

# 데모 실행
await customer_service_demo()

고객 서비스 대화 데모
고객 서비스 대화 시작...

[고객 - 턴 1] 안녕하세요, 제 이름은 김민수이고 주문 관련해서 도움이 필요합니다.
[에이전트] 안녕하세요, 김민수님! 저희 고객센터를 찾아주셔서 감사합니다. 주문 관련해서 어떤 도움이 필요하신가요? 주문 번호나 궁금하신 점을 말씀해주시면 신속하게 도와드리겠습니다.
[메트릭] 메시지: 2, 토큰: 308, 도구 사용: 0회
[고객 정보] {'name': '김민수이고'}
--------------------------------------------------
[고객 - 턴 2] 무선 헤드폰을 주문했는데, 주문번호는 ORD123입니다.
[에이전트] 김민수님, 주문해주신 무선 헤드폰 2개는 현재 “배송중” 상태입니다. 송장번호는 TRK456789로 확인됩니다.

추가로 궁금하신 점이나 요청사항이 있으신가요? 배송 조회나 변경, 반품 등 필요한 서비스를 언제든 말씀해 주세요!
[메트릭] 메시지: 6, 토큰: 804, 도구 사용: 1회
[고객 정보] {'name': '김민수이고', 'recent_order': 'ORD123입니다.'}
--------------------------------------------------
[고객 - 턴 3] 언제 도착하나요? 다음 주에 필요해서요.
[에이전트] 현재 배송이 진행 중이어서 곧 상품을 받으실 수 있을 것 같습니다. 다만, 정확한 배송 도착일은 택배사의 배송 일정에 따라 다를 수 있습니다.

송장번호(TRK456789)로 배송 상황을 자세히 확인해드릴까요, 아니면 직접 배송 조회를 원하시나요? 필요하시면 즉시 배송 추적 결과를 안내드릴 수 있습니다! 어느 쪽이 더 편하실까요?
[메트릭] 메시지: 8, 토큰: 576, 도구 사용: 0회
[고객 정보] {'name': '김민수이고', 'recent_order': 'ORD123입니다.'}
--------------------------------------------------
[고객 - 턴 4] 사실 반품하고 싶은데요.

Error getting response: Error code: 400 - {'error': {'message': 'No tool call found for function call output with call_id call_bkRYrbOEqejzQfozpSZRLsqh.', 'type': 'invalid_request_error', 'param': 'input', 'code': None}}. (request_id: req_2757004c578147a096fd981a06a58558)


응답 오류: Error code: 400 - {'error': {'message': 'No tool call found for function call output with call_id call_bkRYrbOEqejzQfozpSZRLsqh.', 'type': 'invalid_request_error', 'param': 'input', 'code': None}}
[에이전트] 죄송합니다. 기술적인 문제가 발생했습니다. 요청을 다시 말씀해주시거나 문제가 지속되면 지원팀에 연락해주세요.
[메트릭] 메시지: 8, 토큰: 0, 도구 사용: 0회
[고객 정보] {'name': '김민수이고', 'recent_order': 'ORD123입니다.'}
--------------------------------------------------
[고객 - 턴 6] 감사합니다! 참조용 티켓 번호가 뭔가요?


Error getting response: Error code: 400 - {'error': {'message': 'No tool call found for function call output with call_id call_bkRYrbOEqejzQfozpSZRLsqh.', 'type': 'invalid_request_error', 'param': 'input', 'code': None}}. (request_id: req_536cdf28175849ae885a70ca2773b220)


응답 오류: Error code: 400 - {'error': {'message': 'No tool call found for function call output with call_id call_bkRYrbOEqejzQfozpSZRLsqh.', 'type': 'invalid_request_error', 'param': 'input', 'code': None}}
[에이전트] 죄송합니다. 기술적인 문제가 발생했습니다. 요청을 다시 말씀해주시거나 문제가 지속되면 지원팀에 연락해주세요.
[메트릭] 메시지: 8, 토큰: 0, 도구 사용: 0회
[고객 정보] {'name': '김민수이고', 'recent_order': 'ORD123입니다.'}
--------------------------------------------------

대화 요약:
  총 메시지: 8
  수집된 고객 정보: {'name': '김민수이고', 'recent_order': 'ORD123입니다.'}
  핸드오프 준비 완료: True


### 고급 대화 패턴

대화 분기(branching), 병합(merging), 컨텍스트 보존 등의 고급 대화 관리 패턴을 시연한다. 이러한 패턴은 복잡한 대화 흐름을 처리할 때 유용하다.

In [5]:
async def advanced_conversation_patterns():
    """
    고급 대화 관리 패턴을 시연하는 함수이다.
    """
    
    print("\n고급 대화 패턴")
    print("="*50)
    
    agent = Agent(
        name="AdvancedAgent",
        instructions="당신은 대화를 원활하게 이어갈 수 있는 도움이 되는 어시스턴트입니다."
    )
    
    # 패턴 1: 대화 분기 (Branching)
    print("\n1. 대화 분기 (CONVERSATION BRANCHING):")
    print("-" * 40)
    
    result1 = await Runner.run(agent, "일본 여행을 계획하고 있어요.")
    print(f"초기 응답: {result1.final_output[:100]}...")
    
    # 분기 A: 활동에 초점
    branch_a_input = result1.to_input_list() + [{"role": "user", "content": "어떤 활동을 해야 할까요?"}]
    result_a = await Runner.run(agent, branch_a_input)
    print(f"분기 A (활동): {result_a.final_output[:100]}...")
    
    # 분기 B: 예산에 초점 (동일한 시작점에서)
    branch_b_input = result1.to_input_list() + [{"role": "user", "content": "예산은 얼마나 잡아야 할까요?"}]
    result_b = await Runner.run(agent, branch_b_input)
    print(f"분기 B (예산): {result_b.final_output[:100]}...")
    
    return result_a, result_b

# 대화 분기 실행
result_a, result_b = await advanced_conversation_patterns()


고급 대화 패턴

1. 대화 분기 (CONVERSATION BRANCHING):
----------------------------------------
초기 응답: 너무 좋은 계획이네요! 일본 여행을 준비하는 과정도 설레죠. 혹시 어느 지역(도쿄, 오사카, 홋카이도 등)으로 여행할 예정인지 정하셨나요? 아니면 일정, 여행 스타일(맛집, 관광,...
분기 A (활동): 좋은 질문이에요! 일본은 다양한 볼거리와 즐길 거리가 있어서 취향에 맞게 일정을 짤 수 있어요. 아래에 인기 있는 활동들을 지역별로, 그리고 테마별로 정리해볼게요.

---

##...
분기 B (예산): 좋은 질문이에요! 일본 여행 예산은 여행 기간, 여행 도시, 숙박 수준, 여행 스타일(맛집 위주/관광 위주/쇼핑 위주 등), 왕복 항공권 등 다양한 요소에 따라 달라져요. 아래는 ...


In [6]:
# 패턴 2: 대화 병합 (Merging)
print("\n2. 대화 병합 (CONVERSATION MERGING):")
print("-" * 40)

agent = Agent(
    name="AdvancedAgent",
    instructions="당신은 대화를 원활하게 이어갈 수 있는 도움이 되는 어시스턴트입니다."
)

# 양쪽 분기의 인사이트를 결합
merged_input = result_a.to_input_list() + [
    {"role": "user", "content": "예산 조언도 고려해서, 최고의 추천은 뭔가요?"}
]
result_merged = await Runner.run(agent, merged_input)
print(f"병합된 응답: {result_merged.final_output[:150]}...")


2. 대화 병합 (CONVERSATION MERGING):
----------------------------------------
병합된 응답: 아주 좋은 질문이에요! 예산을 함께 고려하면 여행을 훨씬 더 만족스럽게 즐길 수 있습니다.  
일정, 동행 형태(혼자/커플/가족/친구), 여행 기간, 관심사나 선호하는 스타일에 따라 ‘최고의 추천’이 달라질 수 있지만, **가장 효율적이고 만족도가 높은 일본 여행 플랜...


In [7]:
# 패턴 3: 컨텍스트 보존 (Context Preservation)
print("\n3. 컨텍스트 보존 (CONTEXT PRESERVATION):")
print("-" * 40)

conversation = result_merged.to_input_list()

# 사용자 선호도에 대한 컨텍스트 추가
conversation.append({"role": "user", "content": "참고로 저는 채식주의자이고 예술을 좋아해요."})
result_context = await Runner.run(agent, conversation)
print(f"컨텍스트 반영 응답: {result_context.final_output[:150]}...")

print(f"\n최종 대화 길이: {len(result_context.to_input_list())} 메시지")


3. 컨텍스트 보존 (CONTEXT PRESERVATION):
----------------------------------------
컨텍스트 반영 응답: 정말 멋진 여행 스타일이시네요!  
채식주의자시면서 예술을 좋아하신다면, 일본 여행에서도 ‘채식 식당’과 ‘예술 문화 체험’을 중심으로 짜는 것이 만족도가 아주 높으실 거예요.  
아래에 맞춤형으로 추천드릴게요.

---

## ✨ 일본 여행 맞춤 추천 (채식 + 예술...

최종 대화 길이: 8 메시지


### 대화 디버깅 팁

대화 관리 시 발생할 수 있는 문제를 디버깅하기 위한 팁을 정리한다.

  1. 전송 전 항상 conversation_history 길이를 확인하여 토큰 제한을 피한다.
   2. 수동으로 메시지 배열을 구성하지 말고 result.to_input_list()를 사용한다.
   3. 고객/사용자 정보는 대화 기록과 별도로 저장한다.
   4. 턴당 토큰 사용량을 모니터링하여 잘라낼 시점을 예측한다.
   5. 개발 단계에서 대화 분기와 병합을 테스트한다.
   6. 긴 상호작용에 대해 대화 요약을 구현한다.
   7. 구조화된 로깅을 사용하여 대화 상태 변경을 추적한다.

[프로 팁] 프로덕션에서는 데이터베이스 저장을 사용하여
          세션 간 대화를 재개할 수 있도록 대화 영속성을 구현한다.

---

## 2. 프로덕션 대화 관리

이 챕터에서는 프로덕션 환경에서 사용할 수 있는 범용적인 대화 관리 시스템을 구축한다. 자동 대화 기록 관리, 세션 메타데이터, 토큰 제한 관리 등의 기능을 구현한다.

### 학습 내용

- 멀티턴 대화를 위한 ConversationManager 클래스 구축
- 자동 대화 기록 관리 및 트리밍
- 세션 메타데이터 및 컨텍스트 보존
- 대화 상태 관리를 위한 프로덕션 패턴

In [10]:
import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner

### ConversationManager 클래스

범용적인 대화 관리 클래스를 정의한다. 이 클래스는 어떤 에이전트와도 함께 사용할 수 있으며, 대화 기록 관리, 자동 트리밍, 세션 메타데이터 관리 기능을 제공한다.

In [11]:
class ConversationManager:
    """
    컨텍스트 보존을 통해 멀티턴 대화를 관리하는 클래스이다.
    """
    
    def __init__(self, agent, max_history_length=20):
        """
        ConversationManager를 초기화한다.
        
        Args:
            agent: 대화에 사용할 에이전트
            max_history_length: 유지할 최대 대화 기록 길이
        """
        self.agent = agent
        self.conversation_history = []
        self.max_history_length = max_history_length
        self.session_metadata = {}
        self.turn_count = 0
        self.total_tokens = 0
    
    def add_context(self, key, value):
        """세션 컨텍스트에 메타데이터를 추가한다."""
        self.session_metadata[key] = value
    
    async def send_message(self, user_message):
        """
        메시지를 전송하고 대화 상태를 유지한다.
        
        Args:
            user_message: 사용자 메시지
        
        Returns:
            RunResult 객체
        """
        
        # 입력 구성: 대화 기록 + 새 메시지
        if self.conversation_history:
            current_input = self.conversation_history + [
                {"role": "user", "content": user_message}
            ]
        else:
            current_input = user_message
        
        # 에이전트 실행
        result = await Runner.run(self.agent, current_input)
        
        # 대화 기록 업데이트
        self.conversation_history = result.to_input_list()
        
        # 기록이 너무 길면 트리밍
        if len(self.conversation_history) > self.max_history_length:
            # 첫 메시지(보통 시스템/지시문)와 최근 메시지 유지
            self.conversation_history = (
                self.conversation_history[:1] + 
                self.conversation_history[-(self.max_history_length-1):]
            )
        
        # 통계 업데이트
        self.turn_count += 1
        if result.raw_responses:
            self.total_tokens += sum(r.usage.total_tokens for r in result.raw_responses)
        
        return result
    
    def get_conversation_summary(self):
        """분석을 위한 대화 요약을 반환한다."""
        return {
            "total_messages": len(self.conversation_history),
            "turn_count": self.turn_count,
            "total_tokens": self.total_tokens,
            "session_metadata": self.session_metadata,
            "last_role": self.conversation_history[-1].get("role") if self.conversation_history else None
        }
    
    def reset_conversation(self):
        """대화를 초기화한다."""
        self.conversation_history = []
        self.turn_count = 0
        self.total_tokens = 0
        # 메타데이터는 유지

print("ConversationManager 클래스가 정의되었다.")

ConversationManager 클래스가 정의되었다.


### ConversationManager 데모

ConversationManager를 사용하여 고객 지원 대화를 관리하는 예제를 실행한다.

In [12]:
async def conversation_manager_demo():
    """
    ConversationManager를 시연하는 함수이다.
    """
    print("프로덕션 대화 관리자 데모")
    print("=" * 50)
    
    # 에이전트 생성
    support_agent = Agent(
        name="SupportAgent",
        instructions="""
        당신은 고객 지원 에이전트입니다. 대화에서 공유된 고객 세부정보를 기억하고
        개인화된 지원을 제공하세요.
        """
    )

    # 대화 관리자 생성
    conversation = ConversationManager(support_agent)
    conversation.add_context("customer_tier", "프리미엄")
    conversation.add_context("region", "한국")

    # 멀티턴 대화
    print("고객 지원 대화 시작...\n")
    
    response1 = await conversation.send_message("안녕하세요, 계정 로그인에 문제가 있어요.")
    print(f"[고객] 안녕하세요, 계정 로그인에 문제가 있어요.")
    print(f"[에이전트] {response1.final_output}\n")

    response2 = await conversation.send_message("제 사용자명은 hong.gildong@email.com 입니다.")
    print(f"[고객] 제 사용자명은 hong.gildong@email.com 입니다.")
    print(f"[에이전트] {response2.final_output}\n")

    response3 = await conversation.send_message("제 계정 상태를 확인해주시겠어요?")
    print(f"[고객] 제 계정 상태를 확인해주시겠어요?")
    print(f"[에이전트] {response3.final_output}\n")

    return conversation

# 데모 실행
conversation = await conversation_manager_demo()

프로덕션 대화 관리자 데모
고객 지원 대화 시작...

[고객] 안녕하세요, 계정 로그인에 문제가 있어요.
[에이전트] 안녕하세요! 로그인에 문제가 있으시군요. 불편을 드려 죄송합니다. 빠른 해결을 위해 몇 가지 정보를 확인드리겠습니다.

1. 어떤 오류 메시지가 표시되나요? (예: 비밀번호 오류, 계정 없음, 인증 코드 오류 등)
2. 사용하시는 이메일 또는 전화번호는 무엇인가요? (보안을 위해 뒷부분 일부만 입력해 주세요: 예시, ab***@gmail.com)
3. 마지막으로 성공적으로 로그인한 시기는 언제인가요?

이 정보를 알려주시면, 보다 빠르게 도와드릴 수 있습니다!

[고객] 제 사용자명은 hong.gildong@email.com 입니다.
[에이전트] 확인 감사합니다, hong.gildong@email.com 계정으로 로그인하고 계시는군요.

좀 더 정확한 도움을 드리기 위해 아래 질문에 답변해 주시겠어요?  
1. 로그인 시도할 때 화면에 어떤 오류 메시지가 뜨나요?  
2. 혹시 비밀번호를 최근에 변경하신 적이 있나요?  
3. 비밀번호를 잊으신 것인지, 아니면 다른 문제가 있으신 것인지 알려주시면 더 좋겠습니다.

또한, 혹시 가능하시다면 다른 기기나 브라우저에서도 로그인이 되는지 확인해 보실 수 있을까요?  
답변 주시면 바로 도와드리겠습니다!

[고객] 제 계정 상태를 확인해주시겠어요?
[에이전트] 계정 상태 확인을 도와드리겠습니다!  
더 정확한 확인을 위해 몇 가지 절차가 필요합니다.

1. **본인 확인**을 위해 가입하신 이름(또는 닉네임)과 최근 로그인 시기(대략적)도 알려주실 수 있나요?
2. 혹시 고객님의 계정에 등록된 전화번호의 뒷자리(4자리)도 말씀해주실 수 있으신가요? (예: 010-****-1234)

이 정보는 고객님의 계정 보호를 위함이며, 제공해주신 정보는 계정 확인 이후 바로 폐기됩니다.

정보를 확인해 주시면, 계정 상태(정상, 잠김, 비활성화, 로그인 시도 기록 등)를 바로 안내해 드리겠습니다.  
감사합니다

In [13]:
# 대화 요약 출력
print("대화 요약:")
summary = conversation.get_conversation_summary()
print(f"   총 메시지: {summary['total_messages']}")
print(f"   턴 수: {summary['turn_count']}")
print(f"   총 토큰: {summary['total_tokens']}")
print(f"   세션 메타데이터: {summary['session_metadata']}")
print(f"   마지막 역할: {summary['last_role']}")

print(f"\n{'='*50}")
print("데모 완료! 에이전트가 여러 대화 턴에 걸쳐")
print("컨텍스트를 유지하는 것을 확인할 수 있다.")

대화 요약:
   총 메시지: 6
   턴 수: 3
   총 토큰: 1081
   세션 메타데이터: {'customer_tier': '프리미엄', 'region': '한국'}
   마지막 역할: assistant

데모 완료! 에이전트가 여러 대화 턴에 걸쳐
컨텍스트를 유지하는 것을 확인할 수 있다.


### 대화 관리 모범 사례

프로덕션 환경에서 대화를 효과적으로 관리하기 위한 모범 사례를 정리한다.

1. 토큰 관리
   - 대화 기록 길이에 상한선 설정
   - 시스템 메시지는 항상 보존
   - 오래된 메시지는 요약으로 대체 고려

2. 상태 관리
   - 대화 기록과 사용자 정보 분리 저장
   - 세션 메타데이터 활용
   - 핸드오프를 위한 대화 요약 구현

3. 에러 핸들링
   - 폴백 응답 준비
   - 불완전한 도구 호출 처리
   - 토큰 제한 초과 방지

4. 영속성
   - 데이터베이스에 대화 저장
   - 세션 ID로 대화 복원
   - 정기적인 대화 정리 스케줄

### 확장된 대화 관리자 예제

토큰 추적, 대화 내보내기, 요약 생성 등의 추가 기능을 갖춘 확장된 대화 관리자를 구현한다.

In [15]:
class ExtendedConversationManager(ConversationManager):
    """
    추가 기능을 갖춘 확장된 대화 관리자이다.
    """
    
    def __init__(self, agent, max_history_length=20):
        super().__init__(agent, max_history_length)
        self.token_history = []  # 각 턴의 토큰 사용량 추적
    
    async def send_message(self, user_message):
        """메시지를 전송하고 토큰 사용량을 추적한다."""
        result = await super().send_message(user_message)
        
        # 토큰 사용량 기록
        if result.raw_responses:
            turn_tokens = sum(r.usage.total_tokens for r in result.raw_responses)
            self.token_history.append({
                "turn": self.turn_count,
                "tokens": turn_tokens,
                "message_preview": user_message[:50]
            })
        
        return result
    
    def export_conversation(self):
        """대화를 내보내기 가능한 형식으로 반환한다."""
        return {
            "messages": self.conversation_history,
            "metadata": self.session_metadata,
            "statistics": {
                "turn_count": self.turn_count,
                "total_tokens": self.total_tokens,
                "token_history": self.token_history
            }
        }
    
    def get_token_analysis(self):
        """토큰 사용 분석을 반환한다."""
        if not self.token_history:
            return None
        
        tokens = [t["tokens"] for t in self.token_history]
        return {
            "total": sum(tokens),
            "average_per_turn": sum(tokens) / len(tokens),
            "max_turn": max(tokens),
            "min_turn": min(tokens),
            "trend": "증가" if len(tokens) > 1 and tokens[-1] > tokens[0] else "감소 또는 안정"
        }

print("ExtendedConversationManager 클래스가 정의되었다.")

ExtendedConversationManager 클래스가 정의되었다.


In [16]:
# 확장된 관리자 테스트
async def extended_manager_demo():
    agent = Agent(
        name="ExtendedAgent",
        instructions="당신은 친절하고 도움이 되는 어시스턴트입니다."
    )
    
    manager = ExtendedConversationManager(agent)
    manager.add_context("demo", "extended_test")
    
    # 여러 턴 대화
    messages = [
        "안녕하세요!",
        "파이썬이 무엇인지 간단히 설명해주세요.",
        "감사합니다. 배우기 어렵나요?"
    ]
    
    print("=== 확장된 대화 관리자 데모 ===")
    
    for msg in messages:
        result = await manager.send_message(msg)
        print(f"\n[사용자] {msg}")
        print(f"[에이전트] {result.final_output[:100]}...")
    
    return manager

extended_manager = await extended_manager_demo()

=== 확장된 대화 관리자 데모 ===

[사용자] 안녕하세요!
[에이전트] 안녕하세요! 무엇을 도와드릴까요? 😊...

[사용자] 파이썬이 무엇인지 간단히 설명해주세요.
[에이전트] 파이썬(Python)은 쉽고 간단하게 프로그래밍할 수 있도록 설계된 고급 프로그래밍 언어입니다. 문법이 읽기 쉽고 배우기 쉬워서 초보자부터 전문가까지 널리 사용되고 있습니다. 웹 ...

[사용자] 감사합니다. 배우기 어렵나요?
[에이전트] 아니요, 파이썬은 다른 프로그래밍 언어에 비해 배우기 쉬운 편입니다. 다음과 같은 이유로 초보자에게 많이 추천됩니다:

1. **간결한 문법**  
   불필요한 기호나 복잡한 구...


In [17]:
# 토큰 분석 출력
print("\n=== 토큰 분석 ===")
analysis = extended_manager.get_token_analysis()
if analysis:
    print(f"총 토큰: {analysis['total']}")
    print(f"턴당 평균: {analysis['average_per_turn']:.0f}")
    print(f"최대 턴: {analysis['max_turn']}")
    print(f"최소 턴: {analysis['min_turn']}")
    print(f"추세: {analysis['trend']}")


=== 토큰 분석 ===
총 토큰: 676
턴당 평균: 225
최대 턴: 439
최소 턴: 42
추세: 증가


In [18]:
# 대화 내보내기
import json

print("\n=== 대화 내보내기 (요약) ===")
export = extended_manager.export_conversation()
print(f"메시지 수: {len(export['messages'])}")
print(f"메타데이터: {export['metadata']}")
print(f"통계: {json.dumps(export['statistics'], indent=2, ensure_ascii=False)}")


=== 대화 내보내기 (요약) ===
메시지 수: 6
메타데이터: {'demo': 'extended_test'}
통계: {
  "turn_count": 3,
  "total_tokens": 676,
  "token_history": [
    {
      "turn": 1,
      "tokens": 42,
      "message_preview": "안녕하세요!"
    },
    {
      "turn": 2,
      "tokens": 195,
      "message_preview": "파이썬이 무엇인지 간단히 설명해주세요."
    },
    {
      "turn": 3,
      "tokens": 439,
      "message_preview": "감사합니다. 배우기 어렵나요?"
    }
  ]
}


---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK를 사용한 고급 대화 관리 패턴을 다루었다. 다음과 같은 내용을 학습하였다:

1. **CustomerServiceAgent**: 영속적 메모리, 고객 정보 추출, 도구 통합, 토큰 관리를 갖춘 고객 서비스 에이전트
2. **고급 대화 패턴**: 대화 분기(branching), 병합(merging), 컨텍스트 보존
3. **ConversationManager**: 범용적인 멀티턴 대화 관리 클래스
4. **ExtendedConversationManager**: 토큰 추적, 분석, 내보내기 기능을 갖춘 확장 버전
5. **모범 사례**: 토큰 관리, 상태 관리, 에러 핸들링, 영속성

이러한 패턴을 적용하면 복잡한 고객 서비스 시나리오를 처리하고, 대화 컨텍스트를 효과적으로 관리하며, 프로덕션 환경에서 안정적인 대화형 AI 시스템을 구축할 수 있다.